In [1]:
import numpy as np
import pandas as pd

In [2]:
scores = pd.read_csv('scores.csv')
scores.head()

,userId,questionId,score,timestamp
0,1,1,3,964982703
1,1,2,5,964981247
2,1,3,4,964982224
3,1,4,1,964983815
4,1,5,5,964982931


In [3]:
question_bank=pd.read_csv('question_bank.csv')
question_bank.head()

,questionId,question,genres
0,1,When will you hand ____ your assignment?,English
1,2,I am very much __________to meet you.,English
2,3,"If I had helped him, he ____________drowned.",English
3,4,"He has many friends, but _________are good ones.",English
4,5,"_________the rain forests is very important, i...",English


In [4]:
#Creating the rating matrix (rows as question, columns as users)
ratings_mat = np.ndarray(
    shape=(np.max(scores.questionId.values), np.max(scores.userId.values)),
    dtype=np.uint8)
ratings_mat[scores.questionId.values-1, scores.userId.values-1] = scores.score.values
print(ratings_mat)

[[3 0 2 ... 0 0 4]
 [5 1 0 ... 2 3 3]
 [4 1 2 ... 4 1 3]
 ...
 [0 4 2 ... 0 0 2]
 [3 2 1 ... 2 3 1]
 [4 0 3 ... 5 3 4]]


In [5]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T
print(normalised_mat)

[[-0.14 -3.14 -1.14 ... -3.14 -3.14  0.86]
 [ 2.64 -1.36 -2.36 ... -0.36  0.64  0.64]
 [ 1.28 -1.72 -0.72 ...  1.28 -1.72  0.28]
 ...
 [-2.52  1.48 -0.52 ... -2.52 -2.52 -0.52]
 [ 0.34 -0.66 -1.66 ... -0.66  0.34 -1.66]
 [ 1.14 -2.86  0.14 ...  2.14  0.14  1.14]]


In [6]:
#Computing the Singular Value Decomposition (SVD)
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)


AttributeError: 'numpy.ndarray' object has no attribute 'plot'

In [ ]:
#Function to calculate the cosine similarity (sorting by most similar and returning the top N)
def top_cosine_similarity(scores,questionId,top_n=10):
    index = questionId - 1 # starts from 1 in the dataset
    question_row = scores[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', scores, scores))
    similarity = np.dot(question_row, scores.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

In [ ]:
# Function to print top N similar question
def print_similar_question(question_bank, questionId, top_indexes):
    print('Recommendations for {0}: \n'.format(
    question_bank[question_bank.questionId == questionId].question.values[0]))
    for id in top_indexes + 1:
        print(question_bank[question_bank.questionId == id].question.values[0])

In [ ]:
#k-principal components to represent question, questionId to find recommendations, top_n print n results        
k = 50
questionId = 7 # (getting an id from questions.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, questionId, top_n)

In [ ]:
print_similar_question(question_bank, questionId, indexes)